# 빅데이터분석기사 실기 대비
2회차 기출 문제 풀이: 자세한 설명 [링크](https://djccnt15.github.io/dataanalysis/bigdata_certi_03/)

문제 출처
- [빅데이터분석기사 2회 실기 만점 : 문제 복원 및 파이썬 코드 리뷰](https://eatchu.tistory.com/19)
- [빅데이터분석기사 실기 2회 문제 복원 및 후기](https://ha2juo.tistory.com/5)

## 문제 1

주어진 Dataset에서 'CRIM' 값이 가장 큰 10개의 지역을 구하고 10개의 지역의 'CRIM' 값을 그 중 가장 작은 값으로 대체한 후, 'AGE' 컬럼 값이 80 이상인 Row의 'CRIM' 평균값을 구하라

- [데이터 출처](https://www.kaggle.com/code/prasadperera/the-boston-housing-dataset/data)

In [1]:
import pandas as pd

df = pd.read_csv('data/Boston.csv', index_col=0)

In [2]:
df.sort_values(by='crim', ascending=False, inplace=True)
tmp = df[:10]['crim'].min()
df.iloc[:10, 0] = tmp
result = df.loc[df['age']>=80, 'crim'].mean()

print(result)

5.759386625


## 문제 2

주어진 Dataset에서 첫 번째 행 부터 순서대로 80% 까지의 데이터를 추출 후 'total_bedrooms' 컬럼의 중앙값으로 해당 컬럼의 결측치를 대체한 후, 'total_bedrooms' 컬럼의 대치 전후의 표준편차 차이를 구하라

- [데이터 출처](https://www.kaggle.com/datasets/camnugent/california-housing-prices)

In [3]:
import pandas as pd

df = pd.read_csv('data/housing.csv')

In [4]:
df = df.iloc[: round(len(df) * 0.8), :]
std_before = df['total_bedrooms'].std()
df['total_bedrooms'].fillna(value=df['total_bedrooms'].median(), inplace=True)
std_after = df['total_bedrooms'].std()
result = abs(std_after - std_before)

print(result)

1.9751472916456692


## 문제 3

주어진 Dataset의 특정 컬럼의 평균으로부터 1.5 * 표준편차를 벗어나는 영역을 이상치라고 판단하고, 이상치들의 합을 구하라  
(Data가 복원되지 않아 2번 문제와 동일한 데이터 사용)

In [5]:
import pandas as pd

df = pd.read_csv('data/housing.csv')

In [6]:
data = df['total_rooms']
min = df['total_rooms'].mean() - (data.std() * 1.5)
max = df['total_rooms'].mean() + (data.std() * 1.5)
result = df.loc[(df['total_rooms'] > max) | (df['total_rooms'] < min), 'total_rooms'].sum()

print(result)

10707239.0


## 문제 4

E-Commerce_Shipping 데이터를 사용해서 고객이 주문한 물품이 제 시간에 도착여부(Reached.on.Time_Y.N) 예측

### 데이터 읽기

In [7]:
import pandas as pd

df = pd.read_csv('data/E-Commerce_Shipping.csv')

### EDA 진행

In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
 11  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.0+ MB
None


In [9]:
obj_cols = df.select_dtypes(include='object').columns
num_cols = [i for i in df.columns if i not in obj_cols]
num_cols.remove('ID')

In [10]:
print(obj_cols)
print(num_cols)

Index(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], dtype='object')
['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms', 'Reached.on.Time_Y.N']


### 정규화

In [11]:
import numpy as np
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
for col in num_cols:
    df[col] = scaler.fit_transform(np.array(df[col]).reshape(-1, 1))

### One-hot encoding

In [12]:
encoded = pd.get_dummies(df[obj_cols])
df = pd.concat([df, encoded], axis=1)
df.drop(columns=obj_cols, inplace=True)

In [13]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         10999 non-null  int64  
 1   Customer_care_calls        10999 non-null  float64
 2   Customer_rating            10999 non-null  float64
 3   Cost_of_the_Product        10999 non-null  float64
 4   Prior_purchases            10999 non-null  float64
 5   Discount_offered           10999 non-null  float64
 6   Weight_in_gms              10999 non-null  float64
 7   Reached.on.Time_Y.N        10999 non-null  float64
 8   Warehouse_block_A          10999 non-null  uint8  
 9   Warehouse_block_B          10999 non-null  uint8  
 10  Warehouse_block_C          10999 non-null  uint8  
 11  Warehouse_block_D          10999 non-null  uint8  
 12  Warehouse_block_F          10999 non-null  uint8  
 13  Mode_of_Shipment_Flight    10999 non-null  uin

In [14]:
print(df.columns)

Index(['ID', 'Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
       'Prior_purchases', 'Discount_offered', 'Weight_in_gms',
       'Reached.on.Time_Y.N', 'Warehouse_block_A', 'Warehouse_block_B',
       'Warehouse_block_C', 'Warehouse_block_D', 'Warehouse_block_F',
       'Mode_of_Shipment_Flight', 'Mode_of_Shipment_Road',
       'Mode_of_Shipment_Ship', 'Product_importance_high',
       'Product_importance_low', 'Product_importance_medium', 'Gender_F',
       'Gender_M'],
      dtype='object')


### train test set 분리

시험환경과 유사하게 만들어주기 위해 train set과 test set을 분리해준다.

In [15]:
exog = df.drop(columns=['Reached.on.Time_Y.N'])
endog = df.loc[:, 'Reached.on.Time_Y.N']

In [16]:
from sklearn.model_selection import train_test_split

endog_train, endog_test, exog_train, exog_test = train_test_split(endog, exog, test_size=0.2)

### 필요 없는 칼럼 제거

In [17]:
id = exog_test['ID']
exog_train.drop(columns='ID', inplace=True)
exog_test.drop(columns='ID', inplace=True)

### 모델 생성 및 학습

In [18]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X=exog_train, y=endog_train)

LogisticRegression()

### 모델을 활용한 확률 예측

In [19]:
predict = model.predict_proba(exog_test)

print(predict)

[[3.37679121e-02 9.66232088e-01]
 [5.87371615e-03 9.94126284e-01]
 [5.70570284e-01 4.29429716e-01]
 ...
 [5.55102020e-01 4.44897980e-01]
 [6.45161227e-01 3.54838773e-01]
 [9.70752556e-04 9.99029247e-01]]


### 제출용 파일 저장

In [20]:
Reached_on_Time = predict[:, 1]
result = pd.DataFrame({'ID':id, 'probability':Reached_on_Time})

In [21]:
print(result.head())

        ID  probability
491    492     0.966232
4        5     0.994126
5273  5274     0.429430
2025  2026     0.990707
9100  9101     0.480256


In [22]:
result.to_csv('result.csv', index=False)